Now let's generate our own set of Pydantic models of a different FHIR version.

In [1]:
# download the Bundle of R4B StructureDefinitions
!cd spec && ./download_spec_bundle.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5895k  100 5895k    0     0   649k      0  0:00:09  0:00:09 --:--:--  925k
definitions.json.zip: OK
Archive:  definitions.json.zip
  inflating: definitions.json/profiles-types.json  
  inflating: definitions.json/profiles-resources.json  
definitions.json/profiles-types.json -> fhir.r4b.types.json
definitions.json/profiles-resources.json -> fhir.r4b.resources.json


In [3]:
# generate the pydantic classes based on the StructureDefinitions
!typegen --from-bundles spec/fhir.r4b.types.json --from-bundles spec/fhir.r4b.resources.json --out r4b.py

In [3]:
# evaluate the generated classes
from r4b import Condition
c = Condition.parse_file("./Condition.r4b.json")
c.asserter

Reference(id=None, type=None, display='P. van de Heuvel', extension=[], reference='Patient/f001', identifier=None)

In [4]:
# download the Bundle of R5 StructureDefinitions
!cd spec && ./download_spec_bundle.sh r5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6887k  100 6887k    0     0   636k      0  0:00:10  0:00:10 --:--:--  932k29k      0  0:00:20  0:00:05  0:00:15  352k
definitions.json.zip: OK
Archive:  definitions.json.zip
  inflating: profiles-types.json     
  inflating: profiles-resources.json  
profiles-types.json -> fhir.r5.types.json
profiles-resources.json -> fhir.r5.resources.json


In [4]:
# generate the pydantic classes based on the StructureDefinitions
!typegen --from-bundles spec/fhir.r5.types.json --from-bundles spec/fhir.r5.resources.json --out r5.py

In [5]:
from r5 import Condition
c = Condition.parse_file("./Condition.r5.json")
c.asserter # this will fail because asserter is not in the R5 Condition profile

AttributeError: 'Condition' object has no attribute 'asserter'

In [7]:
# this is the new field for asserters and other participants in R5
c.participant

[ConditionParticipant(id=None, actor=Reference(id=None, type=None, display='P. van de Heuvel', extension=[], reference='Patient/f001', identifier=None), function=CodeableConcept(id=None, text=None, coding=[Coding(id=None, system='http://terminology.hl7.org/CodeSystem/provenance-participant-type', version=None, display='Informant', extension=[], userSelected=None, code='informant')], extension=[]), extension=[], modifierExtension=[])]

In [8]:
# now let's generate the mCODE classes which is an ImplementationGuide with a bunch of profiles
!typegen --from-bundles spec/fhir.r4b.types.json --from-bundles spec/fhir.r4b.resources.json --from-bundles spec/fhir.mcode-profiles.json --out mcode.py

In [1]:
# each profile is a class
from mcode import CancerStageGroup

csg = CancerStageGroup.parse_obj({
  "resourceType" : "Observation",
  "status" : "final",
  "code" : {
    "coding" : [
      {
        "system" : "http://loinc.org",
        "code" : "21908-9",
        "display" : "Stage group.clinical Cancer"
      }
    ]
  },
  "subject" : {
    "reference" : "Patient/cancer-patient-john-anyperson"
  },
  "effectiveDateTime" : "2019-04-01",
  "valueCodeableConcept" : {
    "coding" : [
      {
        "system" : "http://cancerstaging.org",
        "code" : "3C",
        "display" : "IIIC"
      }
    ]
  },
  "method" : {
    "coding" : [
      {
        "system" : "http://snomed.info/sct",
        "code" : "897275008",
        "display" : "American Joint Commission on Cancer, Cancer Staging Manual, 8th edition neoplasm staging system (tumor staging)"
      }
    ]
  },
  "hasMember" : [
    {
      "reference" : "Observation/tnm-clinical-primary-tumor-category-cT3"
    },
    {
      "reference" : "Observation/tnm-clinical-regional-nodes-category-cN3"
    },
    {
      "reference" : "Observation/tnm-clinical-distant-metastases-category-cM0"
    }
  ]
})